# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../data/DBTL7/media_descriptions.csv` - media designs for each of the wells
   
   - `../data/DBTL7/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [1]:
import sys
sys.path.append('../media_compiler')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [2]:
CYCLE = 5.1

user_params = {
    'media_file': f'../flaviolin data/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../flaviolin data/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../flaviolin data/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [3]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.05
B1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.05
C1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.05
D1,40.0,4.0,0.074569,20.0,0.710400,3.865275,0.043698,37.793069,1.002822,589.640781,0.000391,0.003163,0.005799,0.012223,0.000270,0.05
E1,40.0,4.0,0.074569,20.0,0.710400,3.865275,0.043698,37.793069,1.002822,589.640781,0.000391,0.003163,0.005799,0.012223,0.000270,0.05


## Create Line Description

In [4]:
df['Line Description'] = df.apply(create_media_description, axis=1)


In [5]:
df['Line Description']

Well
A1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F3    MOPS[mM]: 40.000000, Tricine[mM]: 4.0

## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [6]:
column_order = True if df.index[1][0] == 'B' else False

In [7]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'      
    
def linefunction_row_order_triplicates(row):
    well = row.name

    well_names = list(df.index)
    index = well_names.index(well)
    rem = index%3
    name = f'C{CYCLE}_W{well_names[index-rem]}_{well_names[index-rem+2]}-R{row["Replicate"]}'
    print(name)
    return name
    
    

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]
df['Line Name'] = df.apply(linefunction_row_order_triplicates, axis=1)

# if column_order:
#     df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
# else:
#     df['Line Name'] = df.apply(linefunction_row_order, axis=1)

C5.1_WA1_C1-R1
C5.1_WA1_C1-R2
C5.1_WA1_C1-R3
C5.1_WD1_F1-R1
C5.1_WD1_F1-R2
C5.1_WD1_F1-R3
C5.1_WA2_C2-R1
C5.1_WA2_C2-R2
C5.1_WA2_C2-R3
C5.1_WD2_F2-R1
C5.1_WD2_F2-R2
C5.1_WD2_F2-R3
C5.1_WA3_C3-R1
C5.1_WA3_C3-R2
C5.1_WA3_C3-R3
C5.1_WD3_F3-R1
C5.1_WD3_F3-R2
C5.1_WD3_F3-R3
C5.1_WA4_C4-R1
C5.1_WA4_C4-R2
C5.1_WA4_C4-R3
C5.1_WD4_F4-R1
C5.1_WD4_F4-R2
C5.1_WD4_F4-R3
C5.1_WA5_C5-R1
C5.1_WA5_C5-R2
C5.1_WA5_C5-R3
C5.1_WD5_F5-R1
C5.1_WD5_F5-R2
C5.1_WD5_F5-R3
C5.1_WA6_C6-R1
C5.1_WA6_C6-R2
C5.1_WA6_C6-R3
C5.1_WD6_F6-R1
C5.1_WD6_F6-R2
C5.1_WD6_F6-R3
C5.1_WA7_C7-R1
C5.1_WA7_C7-R2
C5.1_WA7_C7-R3
C5.1_WD7_F7-R1
C5.1_WD7_F7-R2
C5.1_WD7_F7-R3
C5.1_WA8_C8-R1
C5.1_WA8_C8-R2
C5.1_WA8_C8-R3
C5.1_WD8_F8-R1
C5.1_WD8_F8-R2
C5.1_WD8_F8-R3


In [9]:
df

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C5.1_WA1_C1-R1
B1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C5.1_WA1_C1-R2
C1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C5.1_WA1_C1-R3
D1,40.0,4.0,0.074569,20.0,0.710400,3.865275,0.043698,37.793069,1.002822,589.640781,0.000391,0.003163,0.005799,0.012223,0.000270,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C5.1_WD1_F1-R1
E1,40.0,4.0,0.074569,20.0,0.710400,3.865275,0.043698,37.793069,1.002822,589.640781,0.000391,0.003163,0.005799,0.012223,0.000270,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C5.1_WD1_F1-R2
F1,40.0,4.0,0.074569,20.0,0.710400,3.865275,0.043698,37.793069,1.002822,589.640781,0.000391,0.003163,0.005799,0.012223,0.000270,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C5.1_WD1_F1-R3
A2,40.0,4.0,0.079391,20.0,0.805942,3.834796,0.047726,36.907150,1.195513,589.465451,0.000363,0.002957,0.005811,0.008507,0.000865,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C5.1_WA2_C2-R1
B2,40.0,4.0,0.079391,20.0,0.805942,3.834796,0.047726,36.907150,1.195513,589.465451,0.000363,0.002957,0.005811,0.008507,0.000865,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C5.1_WA2_C2-R2
C2,40.0,4.0,0.079391,20.0,0.805942,3.834796,0.047726,36.907150,1.195513,589.465451,0.000363,0.002957,0.005811,0.008507,0.000865,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C5.1_WA2_C2-R3


## Process measurement files

Read measurements file:

In [15]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.9432,0.8961,0.8764,0.9741,0.9844,1.0295,0.8828,0.8887,0.0355,0.0448,0.0450,0.0442
B,0.9167,0.7584,0.8299,0.9209,0.9616,0.9926,0.8466,0.8623,0.0420,0.0409,0.0414,0.0401
C,0.9326,0.9078,0.7107,0.9151,0.9013,0.9822,0.8546,0.8497,0.0354,0.0363,0.0350,0.0353
D,0.5505,0.8029,0.9788,0.9456,0.9562,0.8109,0.8568,0.1594,0.0297,0.0299,0.0296,0.0289
E,0.6155,0.7788,0.9572,0.9858,0.9742,0.8318,0.8888,0.0965,0.0254,0.0251,0.0251,0.0242
F,0.8614,0.8478,0.9496,0.9613,0.9611,0.8805,0.8914,0.0508,0.0218,0.0219,0.0211,0.0217
G,0.0208,0.0212,0.0216,0.0217,0.0216,0.0213,0.0215,0.0214,0.0215,0.0210,0.0207,0.0203
H,0.0217,0.0221,0.0219,0.0219,0.0220,0.0221,0.0218,0.0216,0.0219,0.0218,0.0215,0.0211


In [16]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.5101,0.5074,0.5234,0.6229,0.6863,0.6139,0.6825,0.6269,0.1060,0.1016,0.1009,0.0988
B,0.4961,0.4754,0.5044,0.6222,0.7241,0.6222,0.7789,0.6564,0.1016,0.0992,0.0969,0.0928
C,0.4850,0.4856,0.4296,0.6748,0.7130,0.6753,0.6858,0.6968,0.0934,0.0934,0.0899,0.0883
D,0.3045,0.4898,0.8234,0.7478,0.8376,0.6170,0.6459,0.3751,0.0863,0.0858,0.0836,0.0801
E,0.3375,0.5246,0.9193,0.7846,0.9342,0.6758,0.6957,0.3720,0.0826,0.0798,0.0785,0.0744
F,0.4772,0.5395,0.8331,0.7548,0.8397,0.6804,0.6917,0.3397,0.0770,0.0761,0.0745,0.0726
G,0.0710,0.0749,0.0778,0.0784,0.0794,0.0797,0.0787,0.0791,0.0772,0.0757,0.0731,0.0703
H,0.0711,0.0731,0.0743,0.0770,0.0751,0.0757,0.0740,0.0749,0.0758,0.0737,0.0719,0.0703


### Process OD600

Normalize the data to the control well A9 value (water content):

In [17]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.9077,0.8606,0.8409,0.9386,0.9489,0.9940,0.8473,0.8532,0.0000,0.0093,0.0095,0.0087
B,0.8812,0.7229,0.7944,0.8854,0.9261,0.9571,0.8111,0.8268,0.0065,0.0054,0.0059,0.0046
C,0.8971,0.8723,0.6752,0.8796,0.8658,0.9467,0.8191,0.8142,-0.0001,0.0008,-0.0005,-0.0002
D,0.5150,0.7674,0.9433,0.9101,0.9207,0.7754,0.8213,0.1239,-0.0058,-0.0056,-0.0059,-0.0066
E,0.5800,0.7433,0.9217,0.9503,0.9387,0.7963,0.8533,0.0610,-0.0101,-0.0104,-0.0104,-0.0113
F,0.8259,0.8123,0.9141,0.9258,0.9256,0.8450,0.8559,0.0153,-0.0137,-0.0136,-0.0144,-0.0138
G,-0.0147,-0.0143,-0.0139,-0.0138,-0.0139,-0.0142,-0.0140,-0.0141,-0.0140,-0.0145,-0.0148,-0.0152
H,-0.0138,-0.0134,-0.0136,-0.0136,-0.0135,-0.0134,-0.0137,-0.0139,-0.0136,-0.0137,-0.0140,-0.0144


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [18]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,9.077,8.606,8.409,9.386,9.489,9.940,8.473,8.532
B,8.812,7.229,7.944,8.854,9.261,9.571,8.111,8.268
C,8.971,8.723,6.752,8.796,8.658,9.467,8.191,8.142
D,5.150,7.674,9.433,9.101,9.207,7.754,8.213,1.239
E,5.800,7.433,9.217,9.503,9.387,7.963,8.533,0.610
F,8.259,8.123,9.141,9.258,9.256,8.450,8.559,0.153


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [19]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.4041,0.4014,0.4174,0.5169,0.5803,0.5079,0.5765,0.5209
B,0.3901,0.3694,0.3984,0.5162,0.6181,0.5162,0.6729,0.5504
C,0.3790,0.3796,0.3236,0.5688,0.6070,0.5693,0.5798,0.5908
D,0.1985,0.3838,0.7174,0.6418,0.7316,0.5110,0.5399,0.2691
E,0.2315,0.4186,0.8133,0.6786,0.8282,0.5698,0.5897,0.2660
F,0.3712,0.4335,0.7271,0.6488,0.7337,0.5744,0.5857,0.2337


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [20]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    # df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    # df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

# df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

# df_600.index = df_600['Well']
df_340.index = df_340['Well']

# df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

# df_600.head()

Add measurements to the main dataframe:

In [21]:
# measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
# df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.00247,0.005744,0.006587,0.000057,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C5.1_WA1_C1-R1,0.4041
B1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.00247,0.005744,0.006587,0.000057,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C5.1_WA1_C1-R2,0.3901


## Create EDD Experiment Description File

In [22]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [23]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [24]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [25]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
# df['Measurement Type'] = 'Optical Density'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD600]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [26]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD340]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD

In [27]:
measurement_file = f'{user_params["output_file_path"]}/edd_OD.xlsx'
df['Measurement Type'] = measurOD340

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

In [22]:
df['Kan[g/l]']

Well
A1    0.05
B1    0.05
C1    0.05
D1    0.05
E1    0.05
F1    0.05
A2    0.05
B2    0.05
C2    0.05
D2    0.05
E2    0.05
F2    0.05
A3    0.05
B3    0.05
C3    0.05
D3    0.05
E3    0.05
F3    0.05
A4    0.05
B4    0.05
C4    0.05
D4    0.05
E4    0.05
F4    0.05
A5    0.05
B5    0.05
C5    0.05
D5    0.05
E5    0.05
F5    0.05
A6    0.05
B6    0.05
C6    0.05
D6    0.05
E6    0.05
F6    0.05
A7    0.05
B7    0.05
C7    0.05
D7    0.05
E7    0.05
F7    0.05
A8    0.05
B8    0.05
C8    0.05
D8    0.05
E8    0.05
F8    0.05
Name: Kan[g/l], dtype: float64